## Goal:

One possible way to retrieve winds using wind lidar is using the Doppler beam swing (DBS) scanning strategy. The DBS consists of four slanted observations of the wind. Each one of the observations is from a different azimuth, equally separated by 90 degrees (0, 90, 180, 270). While executing the DBS, the lidar first observes the wind at azimuths of 0 and 180 degrees and then at 90 and 270 degrees. From those observations, the north-south and east-west wind components can be calculated directly.

This example focuses on using lidarwind to retrieve wind speed and direction profiles from the observations collected by the WindCube using the DBS scan strategy. 

## Steps:

1) Downloading sample data from zenodo
1) Reading the DBS files
2) Merging the DBS files
3) Retrieving the wind profiles 
4) Visualising the profiles

In [ ]:
import pooch
import matplotlib.pyplot as plt
import lidarwind as lst

### Step 0: Downloading the sample data

Caching the short DBS sample data

In [ ]:
# 6-beam sample data

file_list = pooch.retrieve(
    url="doi:10.5281/zenodo.7312960/wc_short_dbs.zip",
    known_hash="md5:9cbd93f89052d6c6f4407bcce415e277",
    processor=pooch.Unzip())

file_list = sorted(file_list)

### Step 1 and 2: Reading and merging the DBS 

Here we are going to read all the DBS files. Be careful to provide a list of files that are compatible with each other. Here we also indicate the variables required for processing the DBS files. Finally, the merged dataset is created. 

In [ ]:
var_list = ['azimuth', 'elevation', 'radial_wind_speed', 
            'radial_wind_speed_status', 'measurement_height', 'cnr']

merged_ds = lst.DbsOperations(file_list, var_list).merged_ds

Below, you can see the merged dataset and the variables used during the wind retrieval. You can also see one additional variable, the scan_mean_time. This variable is used to identify observations belonging to the same DBS sequence and is used if the parameter method from  GetWindProperties5Beam is set to single_dbs. 

In [ ]:
merged_ds

### Step 3: Retrieving wind profiles 

Once the merged dataset is created, you can use the dedicated class to retrieve the wind profiles. 

In [ ]:
wind_obj = lst.GetWindProperties5Beam(merged_ds)

As indicated below, you can read the wind profiles directly from the wind_obj (wind object). Since they have different timestamps, it is helpful to resample the data into a regular time grid.

In [ ]:
hor_wind_speed = lst.GetResampledData(wind_obj.hor_wind_speed, time_freq='20s', tolerance=10)
ver_wind_speed = lst.GetResampledData(wind_obj.ver_wind_speed, time_freq='20s', tolerance=10)
hor_wind_dir = lst.GetResampledData(wind_obj.hor_wind_dir, time_freq='20s', tolerance=10)

### Step 4: Visualising the profiles

After resampling, you can use the xarray methods to plot the data. 

In [ ]:
t1 = merged_ds.time.values[0]
t2 = merged_ds.time.values[-1]

hor_wind_speed.resampled.sortby('time_ref').sel(time_ref=slice(t1,t2)).plot(y='range', cmap='turbo')
plt.show()

ver_wind_speed.resampled.sortby('time_ref').sel(time_ref=slice(t1,t2)).plot(y='range', cmap='turbo')
plt.show()

hor_wind_dir.resampled.sortby('time_ref').sel(time_ref=slice(t1,t2)).plot(y='range', cmap='hsv')
plt.show()